# Wyznaczanie wierzchołków pasm materiałów na podłożu GaSb:

Materiał: GaAlSb  
Podłoże: GaSb

In [5]:
material = 'GaAlSb'
ground_material = 'GaSb'

In [23]:
import re
from mendeleev import element
from collections import Counter

def material_type_checker(material):
    component_names = re.findall('[A-Z][^A-Z]*', material) 
    print("Components: ", component_names)

    components = [element(c_name) for c_name in component_names]

    group_counter = Counter([c.group.symbol for c in components])
    material_type = list(group_counter.keys())
    material_type = "-".join(material_type)
    
    return material_type

material_type = material_type_checker(material)

print("Material type:", material_type)

Components:  ['Ga', 'Al', 'Sb']
Material type: IIIA-VA
